The goal of this project is to implement a deep network such that, given as input a series of 2×14×14 tensor, corresponding to pairs of 14 × 14 grayscale images, it predicts for each pair if the first digit is lesser or equal to the second.
The training and test set should be 1, 000 pairs each, and the size of the images allows to run experiments rapidly, even in the VM with a single core and no GPU.


You can generate the data sets to use with the function generate_pair_sets(N) defined in the file dlc_practical_prologue.py.

In [2]:
import dlc_practical_prologue

In [9]:
train_input, train_target, train_classes, test_input, test_target, test_classes = \
    dlc_practical_prologue.generate_pair_sets(100)

|      Name     | Tensor Dimension |   Type  |                  Content                 |
|:-------------:|:----------------:|:-------:|:----------------------------------------:|
|  Train_input  |  N x 2 x 14 x 14 | float32 |                  Images                  |
|  Train_target |         N        |  int64  |          Class to predict €{0,1}         |
| Train_classes |       N x 2      |  int64  | Classes of the two digits € {0, ... ,9 } |
|   Test_input  |  N x 2 x 14 x 14 | float32 |                  Images                  |
|  Test_target  |         N        |  int64  |          Class to predict €{0,1}         |
|  Test_classes |       N x 2      |  int64  | Classes of the two digits € {0, ... ,9 } |